<H1>FBN1: Marfan syndrome (Katzke, 2002)</H1>
<p>Extract phenopackets from the clinical data in <a href="https://pubmed.ncbi.nlm.nih.gov/12203992/" target="__blank">Katzke et al (2002)</a>. Note that we will remove individual B46 from the analysis because only very scant clinical information is provided.</p>

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.3


In [2]:
PMID = "PMID:12203992"
title = "TGGE screening of the entire FBN1 coding sequence in 126 individuals with marfan syndrome and related fibrillinopathies"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
df = pd.read_table("input/katzke_2002.tsv")
df.set_index('Patient', inplace=True)
df.drop("B46",inplace=True)

In [4]:
df

,Exon,Gent,Age,Gender,Skeletal,Ocular,Cardiovascular,Other,FH,Cosegregation,HGVS,Protein
Patient,,,,,,,,,,,,
D15,2,y,25,M,"1,4,8,15",EL,ARD,St,+,NaN,c.184C>T,R62C
D55,2,n,31,M,-,"EL,RD",n,*,-,+,c.184C>T,R62C
D10,3,n,16,F,"7,14,15",EL,n,n,-,+,c.344C>G,S115C
B3,14,e,15,F,"1,4,6","EL,M",ARD,-,-,+,c.1760G>A,C587Y
B1,14,y,8,M,"4,5,7,8,12,14,15","EL,M","ARD,MVP",n,+,+,c.1787G>A,C596Y
D26,15,y,50,F,8,FC,"ARD,AR-dis",St,+,+,c.1960G>A,D654N
B9,16,y,12,M,"1,4,8,10",EL,ARD,*,+,NaN,c.2055C>G,C685W
B19,16,y,3,M,"4,5,6,11,12",EL,ARD,^,^,NaN,c.2055C>G,C685W
D59,16,e,16,F,"1,4,6,11,12","EL,M",NaN,^,^,+,c.2042C>A,S681Y


In [5]:
column_mapper_d = {}

In [6]:
def get_skeletal_items(skel):
    items = str(skel).split(",")
    d = {
        "1": "Tall stature",
        "2": "Pectus carinatum",
        "3": "Pectus excavatum",
        "4": "Arachnodactyly",
        "5": "Dolichostenomelia", # US/LSo0.86 or arm span to height
        "6": "Scoliosis",
        "7": "Limited elbow extension",
        "8": "Pes planus",
        "9": "Protrusio acetabuli",
        "10": "Pectus excavatum",
        "11": "Joint hypermobility",
        "12": "High palate",
        "13": "Malar flattening", #  typical facial appearance in the original publication
        "14": "Dolichocephaly",
        "15": "Enophthalmus"
    }
    results = set()
    for it in items:
        if it in d:
            results.add(d.get(it))
        elif it == "-" or it == "nan" or len(it) == 0:
            continue
        else:
            raise ValueError(f"Could not find \"{it}\" in list")
    if "1" in results and "5" in results:
        results.remove("1") # "1" is implied by "5" and is redundant, so remove it!
    return " ;".join(results)


for pat_id in df.index:
    skeletal = df.at[pat_id, "Skeletal"]
    df.at[pat_id, "Skeletal"] = get_skeletal_items(skel=skeletal)
    #print(df.at[pat_id, "Skeletal"])

In [7]:
df

,Exon,Gent,Age,Gender,Skeletal,Ocular,Cardiovascular,Other,FH,Cosegregation,HGVS,Protein
Patient,,,,,,,,,,,,
D15,2,y,25,M,Arachnodactyly ;Tall stature ;Enophthalmus ;Pes planus,EL,ARD,St,+,NaN,c.184C>T,R62C
D55,2,n,31,M,,"EL,RD",n,*,-,+,c.184C>T,R62C
D10,3,n,16,F,Limited elbow extension ;Dolichocephaly ;Enophthalmus,EL,n,n,-,+,c.344C>G,S115C
B3,14,e,15,F,Arachnodactyly ;Tall stature ;Scoliosis,"EL,M",ARD,-,-,+,c.1760G>A,C587Y
B1,14,y,8,M,Limited elbow extension ;Dolichostenomelia ;Dolichocephaly ;High palate ;Enophthalmus ;Arachnodactyly ;Pes planus,"EL,M","ARD,MVP",n,+,+,c.1787G>A,C596Y
D26,15,y,50,F,Pes planus,FC,"ARD,AR-dis",St,+,+,c.1960G>A,D654N
B9,16,y,12,M,Arachnodactyly ;Tall stature ;Pectus excavatum ;Pes planus,EL,ARD,*,+,NaN,c.2055C>G,C685W
B19,16,y,3,M,Dolichostenomelia ;Joint hypermobility ;High palate ;Arachnodactyly ;Scoliosis,EL,ARD,^,^,NaN,c.2055C>G,C685W
D59,16,e,16,F,Tall stature ;Joint hypermobility ;High palate ;Arachnodactyly ;Scoliosis,"EL,M",NaN,^,^,+,c.2042C>A,S681Y


In [8]:
skelMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d={})
#skelMapper.preview_column(df['Skeletal'])
column_mapper_d['Skeletal'] = skelMapper

In [9]:
ophth_d ={
    "EL": "Ectopia lentis",
    "M": "Myopia",
    "RD": "Retinal detachment",
    "FC": "Flat cornea"
}
ophthMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=ophth_d)
ophthMapper.preview_column(df['Ocular'])
column_mapper_d['Ocular'] = ophthMapper

In [10]:
cv_d ={
    "ARD": "Aortic root aneurysm",
    "MVP":"Mitral valve prolapse",
    'AR-dis':"Ascending aortic dissection",
    'MVP':"Mitral valve prolapse",
    "MR2*": "Mitral regurgitation",
}
cvMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=cv_d)
cvMapper.preview_column(df['Cardiovascular'])
column_mapper_d['Cardiovascular'] = cvMapper

In [11]:
other_d = {
    "St": "Striae distensae",
    "H": "Inguinal hernia",
    "colob": "Lens coloboma"
}
otherMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=other_d)
otherMapper.preview_column(df['Other'])
column_mapper_d['Other'] = otherMapper

In [12]:
transcript = "NM_000138.5"
transcript='NM_000138.5' # FBN1
vvalidator = VariantValidator(genome_build="hg38", transcript=transcript)
variant_d = {}
for v in df["HGVS"].unique():
    var = vvalidator.encode_hgvs(v)
    variant_d[v] = var

varMapper = VariantColumnMapper(variant_d=variant_d,
                                variant_column_name='HGVS', 
                                default_genotype='heterozygous')

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.184C>T/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.344C>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.1760G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.1787G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.1960G>A/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.2055C>G/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.2042C>A/NM_000138.5?content-type=application%2Fjson
https://rest.va

In [13]:
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Gender")
ageMapper = AgeColumnMapper.by_year("Age")
pmid = "PMID:12203992"
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="Patient", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
omim_id = "OMIM:154700"
omim_label = "Marfan syndrome"
mfs = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=mfs)

In [14]:
individuals = encoder.get_individuals()
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_html()))

ID,Level,Category,Message,HPO Term
PMID_12203992_D2,WARNING,REDUNDANT,Tall stature is redundant because of Disproportionate tall stature,Tall stature (HP:0000098)
PMID_12203992_B65,WARNING,REDUNDANT,Tall stature is redundant because of Disproportionate tall stature,Tall stature (HP:0000098)
PMID_12203992_B51,WARNING,REDUNDANT,Tall stature is redundant because of Disproportionate tall stature,Tall stature (HP:0000098)
PMID_12203992_B43,WARNING,REDUNDANT,Tall stature is redundant because of Disproportionate tall stature,Tall stature (HP:0000098)


In [15]:
individuals = cvalidator.get_error_free_individual_list()
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
D15 (MALE; P25Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.184C>T (heterozygous),Arachnodactyly (HP:0001166); Tall stature (HP:0000098); Pes planus (HP:0001763); Ectopia lentis (HP:0001083); Aortic root aneurysm (HP:0002616); Striae distensae (HP:0001065)
D55 (MALE; P31Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.184C>T (heterozygous),Ectopia lentis (HP:0001083); Retinal detachment (HP:0000541)
D10 (FEMALE; P16Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.344C>G (heterozygous),Limited elbow extension (HP:0001377); Dolichocephaly (HP:0000268); Ectopia lentis (HP:0001083)
B3 (FEMALE; P15Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.1760G>A (heterozygous),Arachnodactyly (HP:0001166); Tall stature (HP:0000098); Scoliosis (HP:0002650); Ectopia lentis (HP:0001083); Myopia (HP:0000545); Aortic root aneurysm (HP:0002616)
B1 (MALE; P8Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.1787G>A (heterozygous),Limited elbow extension (HP:0001377); Disproportionate tall stature (HP:0001519); Dolichocephaly (HP:0000268); High palate (HP:0000218); Arachnodactyly (HP:0001166); Pes planus (HP:0001763); Ectopia lentis (HP:0001083); Myopia (HP:0000545); Aortic root aneurysm (HP:0002616); Mitral valve prolapse (HP:0001634)
D26 (FEMALE; P50Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.1960G>A (heterozygous),Pes planus (HP:0001763); Flat cornea (HP:0007720); Aortic root aneurysm (HP:0002616); Ascending aortic dissection (HP:0004933); Striae distensae (HP:0001065)
B9 (MALE; P12Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.2055C>G (heterozygous),Arachnodactyly (HP:0001166); Tall stature (HP:0000098); Pectus excavatum (HP:0000767); Pes planus (HP:0001763); Ectopia lentis (HP:0001083); Aortic root aneurysm (HP:0002616)
B19 (MALE; P3Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.2055C>G (heterozygous),Disproportionate tall stature (HP:0001519); Joint hypermobility (HP:0001382); High palate (HP:0000218); Arachnodactyly (HP:0001166); Scoliosis (HP:0002650); Ectopia lentis (HP:0001083); Aortic root aneurysm (HP:0002616)
D59 (FEMALE; P16Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.2042C>A (heterozygous),Tall stature (HP:0000098); Joint hypermobility (HP:0001382); High palate (HP:0000218); Arachnodactyly (HP:0001166); Scoliosis (HP:0002650); Ectopia lentis (HP:0001083); Myopia (HP:0000545)
D62 (FEMALE; P49Y),Marfan syndrome (OMIM:154700),NM_000138.5:c.2047T>C (heterozygous),Joint hypermobility (HP:0001382); Pectus excavatum (HP:0000767); Arachnodactyly (HP:0001166); Scoliosis (HP:0002650); Pes planus (HP:0001763); Ectopia lentis (HP:0001083); Myopia (HP:0000545); Mitral valve prolapse (HP:0001634); Striae distensae (HP:0001065)


In [16]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              metadata=metadata,
                                              outdir=output_directory)

We output 32 GA4GH phenopackets to the directory phenopackets
